<a href="https://colab.research.google.com/github/vutl/Image-Retrieval/blob/feature%2Fimg-retrieval/Image_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Truy vấn hình ảnh cơ bản**

*   Xây dựng chương trình truy vấn ảnh cơ bản.
*   Phát triển chương trình truy vấn ảnh nâng cao với CLIP model và vector database.
*   (Optional) Thu thập và xử lý dữ liệu nhằm mục đích xây dựng chương trình truy vấn ảnh cá nhân
hóa.





In [1]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

lấy danh sách các class của ảnh trong data

In [ ]:
ROOT = 'data'
CLASS_NAME = sorted(list(os.listdir(f'{ROOT}/train')))

đọc ảnh, resize về kích thước chung (thì
mới áp dụng được các phép đo) và chuyển đổi nó về dạng numpy:

In [ ]:
def read_image_from_path(path, size):
  im = Image.open(path).convert('RGB').resize(size)
  return np.array(im)

def folder_to_images(folder, size):
  list_dir = [folder + '/' + name for name in os.listdir(folder)]
  images_np = np.zeros(shape=(len(list_dir) *size, 3))
  images_path = []
  for i, path in enumerate(list_dir):
    images_np[i] = read_image_from_path(path, size)
    images_path.append(path)
  images_path = np.array(images_path)
  return images_np, images_path

### **Truy vấn hình ảnh với độ đo L1**

In [ ]:
def absolute_difference(query, data):
  axis_batch_size = tuple(range(1, len(data.shape)))
  return np.sum(np.abs(query - data), axis=axis_batch_size)

Tính toán để tính độ tương đồng giữa ảnh input và các hình ảnh trong
bộ dữ liệu. hàm ***get_l1_score*** sẽ trả về ảnh ***query*** và ***ls_path_score*** chứa
danh sách hình ảnh và giá trị độ tương đồng với từng ảnh.

In [ ]:
def get_l1_score(root_img_path, query_path, size):
  query = read_image_from_path(query_path, size)
  ls_path_score = []
  for folder in os.listdir(root_img_path):
    if folder in CLASS_NAME:
      path = root_img_path + folder
      images_np, images_path = folder_to_images(path, size) # mang numpy nhieu anh, paths
      rates = absolute_difference(query, images_np)
      ls_path_score.extend(list(zip(images_path, rates)))
  return query, ls_path_score

Đoạn code này thực hiện quá trình truy xuất hình ảnh bằng cách so sánh một hình ảnh truy vấn với
các hình ảnh trong tập huấn luyện dựa trên điểm L1. Đầu tiên, các hình ảnh được thay đổi cùng kích
thước. Tiếp theo hệ thống sẽ so sánh ảnh truy vấn với các hình ảnh trong thư mục huấn luyện để tính
điểm L1. Sau đó, kết quả truy vấn được trả về là danh sách các đường dẫn chứa hình ảnh và điểm số
tính theo L1. Cuối cùng 5 kết quả tốt nhất sẽ được hiển thị cùng với ảnh truy vấn

In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/Orange_easy/0_100.jpg"
size = (448, 448)
query, ls_path_score = get_l1_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, reverse=False)

In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/African_crocodile/n01697457_18534.JPEG"
size = (448, 448)
query, ls_path_score = get_l1_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, reverse=False)

### **Truy vấn hình ảnh với độ đo L2**

In [ ]:
def mean_square_difference(query, data):
  axis_batch_size = tuple(range(1, len(data.shape)))
  return np.mean((data - query)**2, axis=axis_batch_size)

In [ ]:
def get_l2_score(root_img_path, query_path, size):
  query = read_image_from_path(query_path, size)
  ls_path_score = []
  for folder in os.listdir(root_img_path):
    if folder in CLASS_NAME:
      path = root_img_path + folder
      images_np, images_path = folder_to_images(path, size) # mang numpy nhieu anh, paths
      rates = mean_square_difference(query, images_np)
      ls_path_score.extend(list(zip(images_path, rates)))
  return query, ls_path_score

In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/Orange_easy/0_100.jpg"
size = (448, 448)
query, ls_path_score = get_l2_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, reverse=False)

In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/African_crocodile/n01697457_18534.JPEG"
size = (448, 448)
query, ls_path_score = get_l2_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, reverse=False)

### **Truy vấn hình ảnh với độ đo Cosine Similarity**

In [ ]:
def cosine_similarity(query, data):
  axis_batch_size = tuple(range(1, len(data.shape)))
  query_norm = np.sqrt(np.sum(query**2))
  data_norm = np.sqrt(np.sum(data**2, axis=axis_batch_size))
  return np.sum(data * query, axis=axis_batch_size) / (query_norm * data_norm + np.finfo(float).eps)

In [ ]:
def get_cosine_similarity_score(root_img_path, query_path, size):
  query = read_image_from_path(query_path, size)
  ls_path_score = []
  for folder in os.listdir(root_img_path):
    if folder in CLASS_NAME:
      path = root_img_path + folder
      images_np, images_path = folder_to_images(path, size) # mang numpy nhieu anh, paths
      rates = cosine_similarity(query, images_np)
      ls_path_score.extend(list(zip(images_path, rates)))
  return query, ls_path_score

Để hiển thị kết quả chúng ta sử dụng hàm plot_results(), tuy nhiên ở hàm này chúng ta sẽ sắp xếp giá
trị giảm dần từ lớn đến nhỏ vì với độ đo này thì giá trị càng lớn sẽ càng giống nhau, cho nên chúng ta
sử dụng reverse = True.

In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/Orange_easy/0_100.jpg"
size = (448, 448)
query, ls_path_score = get_cosine_similarity_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, reverse=False)

In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/African_crocodile/n01697457_18534.JPEG"
size = (448, 448)
query, ls_path_score = get_cosine_similarity_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, reverse=False)

### **Truy vấn hình ảnh với độ đo Correlation Coefficient**

In [ ]:
def correlation_coefficient(query, data):
  axis_batch_size = tuple(range(1, len(data.shape)))
  query_mean = query - np.mean(query)
  data_mean = data - np.mean(data, axis=axis_batch_size, keepdims=True)
  query_norm = np.sqrt(np.sum(query_mean**2))
  data_norm = np.sqrt(np.sum(data_mean**2, axis=axis_batch_size))
  return np.sum(data_mean * query_mean, axis=axis_batch_size) / (query_norm * data_norm + np.finfo(float).eps)

In [ ]:
def get_correlation_coefficient_score(root_img_path, query_path, size):
  query = read_image_from_path(query_path, size)
  ls_path_score = []
  for folder in os.listdir(root_img_path):
    if folder in CLASS_NAME:
      path = root_img_path + folder
      images_np, images_path = folder_to_images(path, size) # mang numpy nhieu anh, paths
      rates = correlation_coefficient(query, images_np)
      ls_path_score.extend(list(zip(images_path, rates)))
  return query, ls_path_score

Để hiển thị kết quả chúng ta sử dụng hàm plot_results(), tuy nhiên ở hàm này chúng ta sẽ sắp xếp giá
trị giảm dần từ lớn đến nhỏ vì với độ đo này thì giá trị càng lớn sẽ càng giống nhau, cho nên chúng ta
sử dụng reverse = True

In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/Orange_easy/0_100.jpg"
size = (448, 448)
query, ls_path_score = get_correlation_coefficient_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, reverse=True)

In [ ]:
root_img_path = f"{ROOT}/train/"
query_path = f"{ROOT}/test/African_crocodile/n01697457_18534.JPEG"
size = (448, 448)
query, ls_path_score = get_correlation_coefficient_score(root_img_path, query_path, size)
plot_results(query_path, ls_path_score, reverse=True)

# **Truy vấn hình ảnh nâng cao với Pretrained Deep Learning Model**

*   Thư viện chromadb hỗ trợ việc quản lý và truy xuất dữ liệu hình ảnh hiệu quả (sử dụng thêm với mục đích tạo vector
database)
*   chromadb có thể dùng open-clip-torch để cung cấp khả năng sử dụng mô hình CLIP đã
được đào tạo sẵn, đây là một công cụ mạnh mẽ để phân tích nội dung hình ảnh thông qua học sâu.

In [2]:
!pip install chromadb
!pip install open-clip-torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 114.1 MB/s eta 0:00:00

In [4]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction

Tương tự như các bước ở phần cơ bản trước, nhưng chúng ta sẽ nâng cấp bằng cách thêm một hàm để trích xuất vector
đặc trưng cho mỗi hình ảnh. Mô hình CLIP sẽ được sử dụng để biến đổi hình ảnh thành các vector đặc
trưng đại diện cho nội dung và ngữ cảnh của hình ảnh đó. Sau đó, việc so sánh các hình ảnh không
được thực hiện trực tiếp trên ảnh gốc mà là thông qua việc tính sự tương đồng giữa các vector này.
Đoạn code bên đưới khởi tạo một hàm để trích xuất vector đặc trưng từ một hình sử dụng mô hình CLIP.
Tiếp theo, hàm get_single_image_embedding nhận một hình ảnh làm đầu vào và sử dụng phương thức
_encode_image của OpenCLIPEmbeddingFunction để trích xuất ảnh thành một vector đặc trưng.

In [ ]:
embedding_function = OpenCLIPEmbeddingFunction()

def get_single_image_embedding(image):
  embedding = embedding_function._encode_image(image=image)
  return np.array(embedding)

### **Truy vấn embedding vector với độ đo L1**

Truy vấn embedding vector với độ đo L1 hàm ***get_l1_score*** được nâng cấp lên bằng cách sử dụng
CLIP model để trích xuất vector đặc trưng

In [ ]:
def get_l1_score(root_img_path, query_path, size):
  query = read_image_from_path(query_path, size)
  query_embedding = get_single_image_embedding(query)
  ls_path_score = []
  for folder in os.listdir(root_img_path):
    if folder in CLASS_NAME:
      path = root_img_path + folder
      images_np, images_path = folder_to_images(path, size) # mang numpy nhieu anh, paths
      embedding_list = []
      for idx_img in range(images_np.shape[0]):
        embedding = get_single_image_embedding(images_np[idx_img].astype(np.unit8))
        embedding_list.append(embedding)
      rates = absolute_difference(query_embedding, np.stack(embedding_list))
      ls_path_score.extend(list(zip(images_path, rates)))
  return query, ls_path_score

### **Truy vấn embedding vector với độ đo L2**

In [ ]:
def get_l2_score(root_img_path, query_path, size):
  query = read_image_from_path(query_path, size)
  query_embedding = get_single_image_embedding(query)
  ls_path_score = []
  for folder in os.listdir(root_img_path):
    if folder in CLASS_NAME:
      path = root_img_path + folder
      images_np, images_path = folder_to_images(path, size) # mang numpy nhieu anh, paths
      embedding_list = []
      for idx_img in range(images_np.shape[0]):
        embedding = get_single_image_embedding(images_np[idx_img].astype(np.unit8))
        embedding_list.append(embedding)
      rates = mean_square_difference(query_embedding, np.stack(embedding_list))
      ls_path_score.extend(list(zip(images_path, rates)))
  return query, ls_path_score

### **Truy vấn embedding vector với độ đo Cosine Similarity**

In [ ]:
def get_cosine_similarity_score(root_img_path, query_path, size):
  query = read_image_from_path(query_path, size)
  query_embedding = get_single_image_embedding(query)
  ls_path_score = []
  for folder in os.listdir(root_img_path):
    if folder in CLASS_NAME:
      path = root_img_path + folder
      images_np, images_path = folder_to_images(path, size) # mang numpy nhieu anh, paths
      embedding_list = []
      for idx_img in range(images_np.shape[0]):
        embedding = get_single_image_embedding(images_np[idx_img].astype(np.unit8))
        embedding_list.append(embedding)
      rates = cosine_similarity(query_embedding, np.stack(embedding_list))
      ls_path_score.extend(list(zip(images_path, rates)))
  return query, ls_path_score

### **Truy vấn embedding vector với độ đo Correlation Coefficent**

In [ ]:
def get_correlation_coefficient_score(root_img_path, query_path, size):
  query = read_image_from_path(query_path, size)
  query_embedding = get_single_image_embedding(query)
  ls_path_score = []
  for folder in os.listdir(root_img_path):
    if folder in CLASS_NAME:
      path = root_img_path + folder
      images_np, images_path = folder_to_images(path, size) # mang numpy nhieu anh, paths
      embedding_list = []
      for idx_img in range(images_np.shape[0]):
        embedding = get_single_image_embedding(images_np[idx_img].astype(np.unit8))
        embedding_list.append(embedding)
      rates = correlation_coefficient(query_embedding, np.stack(embedding_list))
      ls_path_score.extend(list(zip(images_path, rates)))
  return query, ls_path_score

## **Tối ưu hoá quá trình truy vấn hình ảnh sử dụng mô hình CLIP và cơ sở dữ liệu vector**

Phương pháp này sẽ sử dụng một cơ sở dữ liệu vector (vector database) để quản lý các
embedding vector, giúp quá trình truy vấn được tối ưu hơn

In [ ]:
def get_files_path(path):
  files_path = []
  for label in CLASS_NAME:
    label_path = path + "/" + label
    filenames = os.listdir(label_path)
    for filename in filenames:
      filepath = label_path + '/' + filename
      files_path.append(filepath)
  return files_path

data_path = f'{ROOT}/train'
files_path = get_files_path(data_path)

### **Truy vấn ảnh với L2 Collection**

Hàm giúp trích xuất và lưu trữ các vector
đặc trưng của ảnh vào một collection đã được tạo (collection -  tập hợp các
vector hoặc tài liệu được chỉ mục và lưu trữ cùng nhau dựa trên một số tiêu chí hoặc đặc điểm chung, dùng
để tổ chức và quản lý dữ liệu)

In [ ]:
def add_embedding(collection, files_path):
  ids = []
  embeddings = []
  for id_filepath, filepath in tqdm(enumerate(files_path)):
    ids.append(f'id_{id_filepath}')
    image = Image.open(filepath)
    embedding = get_single_image_embedding(image=image)
    embeddings.append(embedding)
  collection.add(
      embeddings=embeddings,
      ids=ids
  )

Khởi tạo một client cho cơ sở dữ liệu Chroma và tạo một collection mới với cấu hình
sử dụng L2 để so sánh các embedding vector. Sau đó, gọi hàm add_embedding để thêm các vector đặc trưng của ảnh vào collection này, qua đó tạo điều kiện thuận lợi cho việc truy vấn nhanh chóng và hiệu quả

In [ ]:
#Create a Chroma Client
chroma_client - chromadb.Client()
#Create a collection
l2_collection = chroma_client.get_or_create_collection(name='l2_collection',
                                                      metadata={HNSW_SPACE: "l2"})
add_embedding(collection=l2_collection, files_path=files_path)

Hàm search được định nghĩa để thực hiện truy xuất các ảnh dựa trên embedding của ảnh truy vấn.
Hàm này nhận đường dẫn của ảnh truy vấn, loại collection và số lượng kết quả trả về mong muốn, sau
đó trả về danh sách các kết quả phù hợ

In [ ]:
def search(image_path, collection, n_results):
  query_image = Image.open(image_path)
  query_embedding = get_single_image_embedding(query_image)
  results = collection.query(
      query_embeddings=query_embedding,
      n_results=n_results #how many results to return
  )
  return resutls

In [ ]:
test_path = f'{ROOT}/test'
test_files_path = get_files_path(path=test_path)
test_results = test_files_path[1]
l2_results = search(image_path=test_path, collection=l2_collection, n_results=5)
plot_results(image_path=test_path, files_path=files_path, results=12_results)

### **Truy vấn ảnh với Cosine Similarity Collection**

In [ ]:
#Create a collection
cosine_similarity = chroma.get_or_create_collection(name='cosine_similarity',
                                                      metadata={HNSW_SPACE: "cosine"})
add_embedding(collection=cosine_collection, files_path=files_path)

# **Chương trình Truy Vấn Ảnh Cá Nhân Hóa**

In [5]:
!pip install tqdm
!apt-get update
!apt-get install -y wget
!pip install selenium
!apt-get install -y chromium-browser
!apt-get install -y chromium-chromedriver

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup # For parsing HTML content
from urllib.parse import urljoin, urlparse # For handling URLs
import urllib.request # For making HTTP requests
import time # For handling time-related operations
import os # For interacting with the operating system (relate to dir, folder, file)
from tqdm import tqdm # For displaying progress bars (visualize progress)
import concurrent.futures # For multi-threading
import json # For writing to a text file
from PIL import Image # For handling images
from posix import terminal_size

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [921 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,552 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,841 kB]
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubun

**Thu thập dữ liệu - Crawl URL từ Website**

In [13]:
class UrlScraper:
  #Constructor
  def __init__(self, url_template, max_images=50, max_workers=4):
    self.url_template = url_template #link crawl
    self.max_images = max_images #Max images
    self.max_workers = max_workers #Thread
    self.setup_environment() #Call for set up environment

  #Set up environment
  def setup_environment(self):
    os.environ['PATH'] += ':usr/lib/chronium-browser'
    os.environ['PATH'] += ':usr/lib/chronium-browser/chromedriver'

  def get_url_images(self, term):
    """
    Crawl the urls of images by term

    Parameters:
    term (str): The name of animal, plant, scenery, furniture

    Returns:
    urls (list): List of urls of images
    """

    #Initialize Chrome driver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)

    url = self.url_template.format(search_term=term)
    driver.get(url)

    # Start crawl urls of image like brute force - the same mechanism with this but add some feature
    urls = []
    more_content_available = True

    pbar = tqdm(total=self.max_images, desc=f"Fetching images for {term}") #Set up for visualize progress

    while len(urls) < self.max_images and more_content_available:
      soup  = BeautifulSoup(driver.page_source, "html.parser")
      img_tags = soup.find_all("img")

      for img in img_tags:
        if len(urls) >= self.max_images:
          break
        if 'src' in img.attrs:
          href = img.attrs['src']
          img_path = urljoin(url, href)
          img_path = img_path.replace("_m.jpg", "_b.jpg").replace("_n.jpg", "_b.jpg").replace("_w.jpg", "_b.jpg")
          if img_path ==  "https://combo.staticflickr.com/ap/build/images/getty/IStock_corporate_logo.svg":
            continue
          urls.append(img_path)
          pbar.update(1)

      try:
        load_more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[@id="yui_3_16_0_1_1721642285931_28620"]')))
        load_more_button.click()
        time.sleep(2)
      except:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        new_soup = BeautifulSoup(driver.page_source, "html.parser")
        new_img_tags = new_soup.find_all("img", loading_="lazy")
        if len(new_img_tags) == len(img_tags):
          more_content_available = False
        img_tags = new_img_tags

    pbar.close()
    driver.quit()
    return urls

  def scrape_urls(self, categories):
    """
    Call get_url_images method to get all urls of any object in categories\

    Parameter:
    categories (dictionary): the dict of all object we need to collect image with format
      categories{"name_object": [value1, value2, ...]}

    Returns:
    all_urls (dictionary): Dictionary of urls of images
    """

    all_urls = {category: {} for category in categories}

    #Handle multi-threading for efficient installation
    with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
      futures_to_term = {executor.submit(self.get_url_images, term): (category, term) for category, terms in categories.items() for term in terms}

      for future in tqdm(concurrent.futures.as_completed(futures_to_term), total=len(futures_to_term), desc="Overall Progress"):
        category, term = futures_to_term[future]

        try:
          urls = future.result()
          all_urls[category][term] = urls
          print(f"\nNumber of images retrieved for {term}: {len(urls)}")
        except Exception as exc:
          print(f"\n{term} generated an exception: {exc}")

    return all_urls

  def save_to_file(self, data,  filename):
    """
    Save the data to a JSON file.

    Parameters:
    data (dict): The data to be saved.
    filename (str): The name of the JSON file.

    Returns:
    None

    """
    with open(filename, "w") as file:
      json.dump(data, file, indent=4)
    print(f"Data saved to {filename}")


categories = {
    "animal": ["Monkey", "Elephant", "cows", "Cat", "Dog", "bear", "fox", "Civet", "Pangolins",
               "Rabbit", "Bats", "Whale", "Cock", "Owl", "flamingo", "Lizard", "Turtle", "Snake",
               "Frog", "Fish", "shrimp", "Crab", "Snail", "Coral", "Jellyfish", "Butterfly", "Flies",
               "Mosquito", "Ants", "Cockroaches", "Spider", "scorpion", "tiger", "bird", "horse",
               "pig", "Alligator", "Alpaca", "Anteater", "donkey", "Bee", "Buffalo", "Camel"],

    "plant": ["Bamboo", "Apple", "Apricot", "Banana", "Bean", "Wildflower", "Flower",
              "Mushroom", "Weed", "Fern", "Reed", "Shrub", "Moss", "Grass", "Palmtree", "Corn",
              "Tulip", "Rose", "Clove", "Dogwood", "Durian", "Ferns", "Fig", "Flax", "Frangipani",
              "Lantana", "Hibiscus", "Bougainvillea", "Pea", "OrchidTree", "RangoonCreeper",
              "Jackfruit", "Cottonplant", "Cornelainetree", "Coffeplant", "Coconut", "wheat",
              "watermelon", "radish", "carrot"],

    "furniture": ["bed", "cabinet", "chair", "chests", "clock", "desks", "table", "Piano",
                  "Bookcase", "Umbrella", "Clothes", "cart", "sofa", "ball", "spoon", "Bowl", "fridge",
                  "pan", "book"],

    "scenery": ["Cliff", "Bay", "Coast", "Mountains", "Forests", "Waterbodies", "Lake",
                "desert", "Farmland", "river", "hedges", "plain", "sky", "cave", "cloud", "flowergarden",
                "glacier", "grassland", "horizon", "lighthouse", "plateau", "savannah", "valley",
                "volcano", "waterfall"]
}

urltopic = {"flickr": "https://www.flickr.com/search/?text={search_term}"}
scraper = UrlScraper(url_template=urltopic["flickr"], max_images=20, max_workers=5)
image_urls = scraper.scrape_urls(categories)
scraper.save_to_file(image_urls, 'image_urls.json')

Fetching images for cows:   5%|▌         | 1/20 [00:01<00:20,  1.09s/it]

Fetching images for Dog:   0%|          | 0/20 [00:00<?, ?it/s]

Fetching images for Dog:   5%|▌         | 1/20 [00:02<00:43,  2.26s/it]


Fetching images for Monkey:   0%|          | 0/20 [00:00<?, ?it/s]



Fetching images for Elephant:   0%|          | 0/20 [00:00<?, ?it/s]




Fetching images for Cat:   0%|          | 0/20 [00:00<?, ?it/s]


Fetching images for Monkey:   5%|▌         | 1/20 [00:01<00:30,  1.62s/it]



Fetching images for Elephant:   5%|▌         | 1/20 [00:01<00:34,  1.80s/it]




Overall Progress:   1%|          | 1/127 [00:43<1:32:21, 43.98s/it]


Number of images retrieved for cows: 20



Overall Progress:   2%|▏         | 2/127 [00:48<43:02, 20.66s/it]  


Number of images retrieved for Dog: 20




Fetching images for fox:   0%|          | 0/20 [00:00<?, ?it/s]

Overall Progress:   2%|▏         | 3/127 [00:52<26:51, 13.00s/it]

...


Number of images retrieved for valley: 20



Overall Progress:  99%|█████████▉| 126/127 [10:58<00:05,  5.01s/it]


Number of images retrieved for volcano: 20


Overall Progress: 100%|██████████| 127/127 [11:02<00:00,  5.22s/it]


Number of images retrieved for waterfall: 20
Data saved to image_urls.json


**Thu thập dữ liệu - Crawl ảnh từ URL**

In [17]:
import os
import json
import time
import urllib.request
from collections import defaultdict
from tqdm import tqdm
import concurrent.futures
from urllib.parse import urlparse

class ImageDownloader:
    def __init__(self, json_file, download_dir='Dataset', max_workers=4, delay=1):
        self.json_file = json_file  # File chứa các URL hình ảnh ở định dạng JSON
        self.download_dir = download_dir  # Tên thư mục để lưu trữ hình ảnh
        self.max_workers = max_workers  # Số luồng thực thi
        self.delay = delay  # Độ trễ giữa các request để tránh việc gửi quá nhiều yêu cầu tới máy chủ
        self.filename = set()  # Để lưu trữ các đường dẫn file đã tải xuống
        self.setup_directory()  # Thiết lập cấu trúc thư mục

    def setup_directory(self):
        if not os.path.exists(self.download_dir):
            os.makedirs(self.download_dir)

    def read_json(self):
        """
        Đọc file JSON và trả về dữ liệu.

        Returns:
        data (dict): Dữ liệu được đọc từ file JSON.
        """
        with open(self.json_file, 'r') as file:
            data = json.load(file)
        return data

    def is_valid_url(self, url):
        """
        Kiểm tra xem URL có hợp lệ hay không.

        Parameters:
        url (str): URL cần kiểm tra.

        Returns:
        bool: True nếu URL hợp lệ, False nếu không hợp lệ.
        """
        try:
            with urllib.request.urlopen(url) as response:
                if response.status == 200 and 'image' in response.info().get_content_type():
                    return True
        except Exception:
            return False

    def download_image(self, url, category, term, pbar):
        """
        Tải hình ảnh từ URL đã cho.

        Parameters:
        url (str): URL của hình ảnh cần tải xuống.
        category (str): Danh mục của hình ảnh.
        term (str): Từ khóa hoặc thuật ngữ liên quan đến hình ảnh.
        pbar (tqdm): Đối tượng thanh tiến trình.

        Returns:
        str: Thông báo về trạng thái của quá trình tải xuống.
        """
        if not self.is_valid_url(url):
            pbar.update(1)
            return f"Invalid URL: {url}"

        category_dir = os.path.join(self.download_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)

        term_dir = os.path.join(category_dir, term)
        if not os.path.exists(term_dir):
            os.makedirs(term_dir)

        filename = os.path.join(term_dir, os.path.basename(urlparse(url).path))
        self.filename.add(filename)  # Ghi lại đường dẫn file đã tải xuống

        try:
            urllib.request.urlretrieve(url, filename)
            pbar.update(1)
            return f"Downloaded: {url}"
        except Exception as e:
            pbar.update(1)
            return f"Failed to download {url}: {str(e)}"

    def download_images(self):
        """
        Tải xuống các hình ảnh từ file JSON.

        Returns:
        None
        """
        data = self.read_json()
        download_tasks = []

        total_images = sum(len(urls) for terms in data.values() for urls in terms.values())
        with tqdm(total=total_images, desc="Downloading images") as pbar:
            with concurrent.futures.ThreadPoolExecutor(max_workers=self.max_workers) as executor:
                for category, terms in data.items():
                    for term, urls in terms.items():
                        for url in urls:
                            download_tasks.append(
                                executor.submit(self.download_image, url, category, term, pbar)
                            )
                        time.sleep(self.delay)  # Thêm độ trễ để tránh gửi quá nhiều yêu cầu tới máy chủ

            for future in concurrent.futures.as_completed(download_tasks):
                print(future.result())

        self.export_filename()

    def export_filename(self):
        """
        Xuất các đường dẫn file đã tải xuống vào một file văn bản.

        Returns:
        None
        """
        with open('filename.txt', 'w') as file:
            for filename in sorted(self.filename):
                file.write(f"{filename}\n")

In [18]:
downloader = ImageDownloader(json_file='image_urls.json', download_dir='Dataset', max_workers=4, delay=1)
downloader.download_images()
downloader.export_filename()

KeyboardInterrupt: 

**Xử lí dữ liệu - Làm sạch bộ dữ liệu**

In [ ]:
from google.colab import drive
drive.mount('content/drive/')

In [ ]:
def check_and_preprocess_images(image_dir):
  """
  Check and preprocess images in the specified directory.

  Parameters:
  image_dir (str): The directory containing the images to be checked and preprocessed.

  Returns:

  None
  """
  for root, _, files in os.walk(image_dir):
    for filename in files:
      filepath = os.path.join(root, file)
      try:
        with Image.open(filepath) as img:
          #Check if image is smaller than 50x50 pixels
          if img.size[0] < 50 or img.size[1] < 50:
            os.remove(filepath)
            print(f"Deleted {file_path}: Image too small ({img.size[0]}x{img.size[1]})")
            continue

          #Convert non-RGB images to RGB
          if img.mode != 'RGB':
            img = img.convert('RGB')
            img.save(filepath)
            print(f"Converted {file_path} to RGB")

      except Exception as e:
        #If file is not an image, delete it
        os.remove(filepath)
        print(f"Deleted {file_path}: Not an image or corrupted file ({str(e)})")


check_and_preprocess_images('Dataset')

In [ ]:
!zip -r /content/drive/MyDrive/Clean_Dataset.zip Dataset

**Xử lí dữ liệu - Tổ chức cấu trúc folder**

In [ ]:
!gdown --id 1--6fe48D9ydnTpLV1GKKqJ0pqpOXB3z_

In [ ]:
!unzip Clean_Dataset

In [ ]:
import os
import shutil
from collections import defaultdict

# Define the source and target directories
source_dir = "Dataset"
train_dir = "data/train"
test_dir = "data/test"

# Create the target directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Initialize a dictionary to hold file paths for each class
class_files = defaultdict(list)

# Read the file paths from the text file
with open('filename.txt', 'r') as file:
    lines = file.readlines()
    for line in lines:
        line = line.strip()
        if line:
            # Extract the class name from the path
            parts = line.split('/')
            class_name = parts[2]  # Structure Dataset/category/class/image.jpg
            class_files[class_name].append(line)

# Move images to the train and test directories
for class_name, files in class_files.items():
    # Create the train and test directories for the class
    train_class_dir = os.path.join(train_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    # Move 19 images to train and 1 image to test
    for i, file_path in enumerate(files):
        if i == 0:
            shutil.copy(file_path, test_class_dir)
        elif i < 20:
            shutil.copy(file_path, train_class_dir)

print("Dataset organization complete!")

In [ ]:
!zip -r /content/drive/MyDrive/data.zip data